In [1]:
import numpy as np
import pandas as pd

distinct_features = ["By Owner",
                     "Exclusive",
                     "Sublet / Lease-Break",
                     "No Fee",
                     "Reduced Fee",
                     "Short Term Allowed",
                     "Furnished",
                     "Laundry In Unit",
                     "Private Outdoor Space",
                     "Parking Space",
                     "Cats Allowed",
                     "Dogs Allowed",
                     "Doorman",
                     "Elevator",
                     "Fitness Center",
                     "Laundry In Building",
                     "Common Outdoor Space",
                     "Storage Facility"]

def load_data(filename):
    with open(filename) as f:
        data = pd.read_json(f)

    dist = data.features.apply(
        lambda x: pd.Series(map(lambda z: 1 if (z in x) else 0, distinct_features) +
                            [list(np.setdiff1d(x, distinct_features))]))
    dist.columns = distinct_features + ["UNIQUES"]

    data = data.join(dist)

    man_counts = pd.DataFrame(data.manager_id.value_counts())
    man_counts["manager count"] = man_counts["manager_id"]
    man_counts["manager_id"] = man_counts.index

    data = pd.merge(data, man_counts, on="manager_id")

    return data


train_data = pd.read_json("~/two-sigma/data/processed_train.json")

In [4]:
train_data.columns

Index([u'By Owner', u'Cats Allowed', u'Common Outdoor Space', u'Dogs Allowed',
       u'Doorman', u'Elevator', u'Exclusive', u'Fitness Center', u'Furnished',
       u'Laundry In Building', u'Laundry In Unit', u'No Fee', u'Parking Space',
       u'Private Outdoor Space', u'Reduced Fee', u'Short Term Allowed',
       u'Storage Facility', u'Sublet / Lease-Break', u'avg_brightness',
       u'avg_imagesize', u'avg_luminance', u'bathrooms', u'bedrooms',
       u'building_id', u'created', u'description', u'description_sentiment',
       u'display_address', u'dist_count', u'features', u'img_quantity',
       u'interest_level', u'latitude', u'listing_id', u'longitude',
       u'manager count', u'manager_id', u'photos', u'price', u'street_address',
       u'unique_count', u'word_count'],
      dtype='object')

In [158]:
import numpy as np
train_data['LogScaledPrice'] = np.log(train_data['price'])

In [213]:
train_data['Rooms'] = train_data['bedrooms'] + train_data['bathrooms']

In [227]:
train_data = train_data.fillna(0)

In [228]:
df = train_data[['LogScaledPrice','avg_imagesize','word_count','avg_luminance','description_sentiment','Rooms']]

df1 = train_data['interest_level']

In [235]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, df1, test_size=0.2, random_state=42)

In [236]:
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier(n_estimators=65,max_features = 'sqrt',oob_score=True,min_samples_leaf = 400,n_jobs = 1)
clf1 = clf1.fit(X_train,y_train)
clf1.score(X_train,y_train)

0.69468858438236114

In [237]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,clf1.predict(X_test))

0.69466112855840345